# 本地调用tool工具-使用AppBuilderEventHandler简化tool_call操作

基于multi_tool_call的示例，我们可以使用Client应用来执行tool_call操作，完成指定的命令，但是需要自己配置client的思考与运行流程，较为繁琐。SDK提供了使用AppBuilderEventHandler简化tool_call操作的功能

##### 配置运行环境&导入Client应用

In [4]:
import os
import appbuilder


# AppBuilder Token
os.environ["APPBUILDER_TOKEN"] = "your api key"

# 应用为：智能问题解决者
app_id = "b9473e78-754b-463a-916b-f0a9097a8e5f"
app_client = appbuilder.AppBuilderClient(app_id)
conversation_id = app_client.create_conversation()

##### 继承AppBuilderEventHandler类实现一个Agent框架的定义

In [5]:
from appbuilder.core.console.appbuilder_client.event_handler import AppBuilderEventHandler
class MyEventHandler(AppBuilderEventHandler):
    def execute_local_command(self, cmd: str):
        import subprocess
        try:
            result = subprocess.check_output(cmd, shell=True).decode("utf-8")
            if result.strip() == "":
                return "命令执行成功，无返回值"
            return result
        except Exception as e:
            return str(e)
    
    def interrupt(self, run_context, run_response):
        thought = run_context.current_thought
        # 绿色打印
        print("\033[1;32m", "-> Agent 中间思考: ", thought, "\033[0m")

        tool_output = []
        for tool_call in run_context.current_tool_calls:
            tool_call_id = tool_call.id
            tool_res = self.execute_local_command(
                **tool_call.function.arguments)
            # 蓝色打印
            print("\033[1;34m", "-> 本地ToolCall结果: \n", tool_res, "\033[0m\n")
            tool_output.append(
                {
                    "tool_call_id": tool_call_id,
                    "output": tool_res
                }
            )
        return tool_output
    
    def success(self, run_context, run_response):
        print("\n\033[1;31m","-> Agent 非流式回答: \n", run_response.answer, "\033[0m")

##### 定义本地的tools工具

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_local_command",
            "description": "可以在bash环境中，执行输入的指令，注意，一次只能执行一个原子命令。例如：ls",
            "parameters": {
                "type": "object",
                "properties": {
                    "cmd": {
                        "type": "string",
                        "description": "需要执行的指令",
                    },
                },
                "required": ["cmd"],
            },
        },
    }
]

##### 运行Client实现实现链路的调用

In [ ]:
with app_client.run_with_handler(
        conversation_id = conversation_id,
        query = "请问当前文件夹下有哪些文件？如果没有test.txt文件，请新建一个test.txt文件，内容为：Hello World！",
        tools = tools,
        event_handler = MyEventHandler(),
    ) as run:
        run.until_done()

**output**
```
 -> Agent 中间思考:  首先，我需要使用execute_local_command工具来执行'ls'命令，列出当前文件夹下的所有文件。然后，我需要检查输出中是否存在test.txt文件。如果不存在，我将再次使用execute_local_command工具来执行'echo "Hello World" > test.txt'命令，以创建并写入test.txt文件。 
 -> 本地ToolCall结果: 
 multi_tool_call.ipynb
multi_tool_call.py
multi_tool_call_with_handler.ipynb
multi_tool_call_with_handler.py
sdk_ knowledgebase.ipynb
sdk_trace.ipynb
simple_tool_call.ipynb
simple_tool_call.py
tmp.log
黑神话(悟空).pdf
 

 -> Agent 中间思考:  根据execute_local_command工具的返回结果，当前文件夹下并没有test.txt文件。因此，我需要使用execute_local_command工具来执行'echo "Hello World" > test.txt'命令，以创建并写入test.txt文件。 
 -> 本地ToolCall结果: 
 命令执行成功，无返回值 


 -> Agent 非流式回答: 
 当前文件夹下的文件包括：

- multi_tool_call.ipynb
- multi_tool_call.py
- multi_tool_call_with_handler.ipynb
...
- tmp.log
- 黑神话(悟空).pdf

经过检查，发现当前文件夹下**不存在**test.txt文件。因此，已经为您新建了一个test.txt文件，并写入了内容“Hello World！”。 
```

- 使用AppBuilderEventHandler架构可以简化client的交互方式